In [ ]:
#Import Librairies 
!pip install nltk
!pip install spacy==2.3.5
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
!pip install pyresparser
!pip install -U sentence-transformers 
!pip install pandas
!pip install -U sentence-transformers
import nltk
nltk.download('stopwords')
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification
import os
import torch.nn.functional as F
from transformers import pipeline, set_seed
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
import torch
from sentence_transformers import SentenceTransformer, util
import python_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 75.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Upload CV 
def preprocess(path):
##file format should be in .txt , .docx or .pdf only
    filed=input()
    return(filed)
def parser(path):
#CV Parsing : name , email, mobile ,skills, experience, degree
  from pyresparser import ResumeParser
  data = ResumeParser(path).get_extracted_data()
  sk=data['skills']
  print('candidate skills are=',sk)
  with open("skill.txt", "w") as f:
      for s in sk:
        f.write(str(s) +"\n")
#Réferentiel de sills : 
  return(sk)
def ref(back1,front1):
    back1=['Python','tkinter','Scikit-learn', 'Beautifulsoup', 'Opencv', 'Segmentation', 'Spacy', 'Oracle', 'Nltk', 'Pdf', 'Docker', 'Keras']
    with open("back.txt", "w") as f:
       for s in back1:
           f.write(str(s) +"\n")
    front1=['angular','java']
    with open("front.txt", "w") as f:
        for s in front1:
            f.write(str(s) +"\n")
    return(back1,front1)

/content/CV_HeZhang.pdf


/usr/local/lib/python3.9/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


candidate skills are= ['Analysis', 'Analytics', 'Perl', 'Data analytics', 'Writing', 'Segmentation', 'Machine learning', 'Matlab', 'International', 'Sql', 'Engineering', 'C', 'Matrix', 'Research', 'Content', 'Analytical', 'Travel', 'Updates', 'Video', 'Technical', 'Transactions', 'Programming', 'Algorithms', 'Mining', 'Pattern', 'C++', 'Presentation', 'Email', 'Python', 'System', 'Datasets', 'Supervisor', 'English', 'Communication', 'Reports', 'Mobile', 'Computer science', 'Marketing', 'Javascript']


(['Python',
  'tkinter',
  'Scikit-learn',
  'Beautifulsoup',
  'Opencv',
  'Segmentation',
  'Spacy',
  'Oracle',
  'Nltk',
  'Pdf',
  'Docker',
  'Keras'],
 ['angular', 'java'])

In [ ]:
#Matching between the extracted CV skills and referentiel skills using multilingual BERT and similarity sentence task as an output layer 
#Mean Pooling - Take attention mask into account for correct aver
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Sentences we want sentence embeddings for
sentences = ['/content/skill.txt','/content/back.txt','/content/front.txt']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/distiluse-base-multilingual-cased-v2')
model = AutoModel.from_pretrained('sentence-transformers/distiluse-base-multilingual-cased-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')


# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
# compute similarity scores of two embeddings
cosine_scores1 = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1]) 
cosine_scores2 = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[2]) 
print("Back:", cosine_scores1.item())
print("Front:", cosine_scores2.item())


Back: 0.8961853384971619
Front: 0.9120257496833801
